In [1]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 3.9 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from torch.nn.parallel import DataParallel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import datasets

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
model = DataParallel(model)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
new_tokens = ['<H>', '<R>', '<T>', '[SEP]']
new_tokens_vocab = {}
new_tokens_vocab['additional_special_tokens'] = []
for idx, t in enumerate(new_tokens):
    new_tokens_vocab['additional_special_tokens'].append(t)
num_added_toks = tokenizer.add_special_tokens(new_tokens_vocab)

tokenizer.add_tokens("[MASK]")
tokenizer.mask_token = "[MASK]"
tokenizer.mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")

In [4]:
class WebNLGDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "translate from Graph to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # preprocess the input graph
        try:
            triples = item['modified_triple_sets']['mtriple_set']
            input_text = self.prefix
            for outer_list in triples:
                for triple in outer_list:
                    triple_txt = triple.split("|")
                    input_text += " <H> " + triple_txt[0] + " <R> " + triple_txt[1] + " <T> " + triple_txt[2]
        except (KeyError, IndexError):
            print("1")
            print(item['original_triple_sets']['otriple_set'])
            print(item['original_triple_sets']['otriple_set'][0])
            print(triples)
            input_text = self.prefix
        # preprocess the target text
        try:
            target_text = item['text'][0]
        except (KeyError, IndexError):
            print("2")
            print(item)
            #print(item['original_triple_sets']['otriple_set'])
            target_text = ""
        #print(item)
        #print(input_text)
        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        #print(input_text)
        #print(target_text)
        return input_ids.squeeze(0), target_ids.squeeze(0)


In [5]:
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128
tokenizer.model_max_length = MAX_INPUT_LENGTH
model.module.config.max_length = MAX_TARGET_LENGTH

# set up the device (GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

DataParallel(
  (module): T5ForConditionalGeneration(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=768, out_features=2048, bias=False)
                (wi_1): Linear

In [6]:
import pickle

with open('/kaggle/input/models/final_dataset.pkl', 'rb') as f:
  final_dataset = pickle.load(f)

In [7]:
import random

random.Random(4).shuffle(final_dataset)

In [8]:
dataset = final_dataset[:700]
dataset_val = final_dataset[700:900]
dataset_test = final_dataset[900:]

In [9]:
new_dataset = []

for i in range(len(dataset)):
  sample = dataset[i]
  target_list = sample['lex']['text']
  reason = sample['reasoning']
  mod_text_list = []
  for a in target_list:
    b = reason + " [SEP]" + a
    mod_text_list.append(b)
  sample['text'] = mod_text_list
  new_dataset.append(sample)

In [10]:
new_dataset_val = []

for i in range(len(dataset_val)):
  sample = dataset[i]
  target_list = sample['lex']['text']
  reason = sample['reasoning']
  mod_text_list = []
  for a in target_list:
    b = reason + " [SEP]" + a
    mod_text_list.append(b)
  sample['text'] = mod_text_list
  new_dataset_val.append(sample)

In [11]:
new_dataset_test = []

for i in range(len(dataset_test)):
  sample = dataset[i]
  target_list = sample['lex']['text']
  reason = sample['reasoning']
  mod_text_list = []
  for a in target_list:
    b = reason + " [SEP]" + a
    mod_text_list.append(b)
  sample['text'] = mod_text_list
  new_dataset_test.append(sample)

In [12]:
train_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=new_dataset))

In [13]:
val_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=new_dataset_val))
test_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=new_dataset_test))

In [14]:
dataset_train = WebNLGDataset(train_dataset)
dataset_val = WebNLGDataset(val_dataset)
dataset_test = WebNLGDataset(test_dataset)

In [15]:
# load the WebNLG dataset
#dataset = load_dataset('web_nlg', 'release_v3.0_en')['train']
#dataset_val = load_dataset('web_nlg', 'release_v3.0_en')['dev']

In [16]:
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss


# Adaptive pretraining

For STA, we fine-tuned the PLMs on a small amount of labeled data from the target task using a maximum likelihood estimation (MLE) objective. This involves training the model to maximize the likelihood of generating the correct output given the input graph and labeled data. This process helps to further adapt the PLM to the specific requirements of the target task and improve its performance on that task.

In [ ]:
import random

pretrain_texts = []
for sample in dataset:
    try:
        text = sample['text'][0]
        pretrain_texts.append(text)
    except (KeyError, IndexError):
        continue

tokenized_inputs = tokenizer(pretrain_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']

pretrain_data = torch.utils.data.TensorDataset(input_ids, attention_mask)

pretrain_loader = torch.utils.data.DataLoader(pretrain_data, batch_size=int(60), shuffle=True)

pretrain_optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
pretrain_criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

pretrain_epochs = 10  # Set the number of pre-training epochs
masking_prob = 0.15  # Probability of masking a token


# Prepare validation data
val_texts = []
for sample in dataset_val:
    try:
        text = sample['text'][0]
        val_texts.append(text)
    except (KeyError, IndexError):
        continue

tokenized_inputs_val = tokenizer(val_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
input_ids_val = tokenized_inputs_val['input_ids']
attention_mask_val = tokenized_inputs_val['attention_mask']

val_data = TensorDataset(input_ids_val, attention_mask_val)

val_loader = DataLoader(val_data, batch_size=int(60), shuffle=True)

early_stopping = EarlyStopping(patience=3, verbose=True)

if tokenizer.mask_token is None:
    # Manually set a mask token if not already defined
    tokenizer.add_tokens("[MASK]")
    tokenizer.mask_token = "[MASK]"
    tokenizer.mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")

for epoch in range(pretrain_epochs):
    running_loss = 0.0
    for inputs, attention_mask in pretrain_loader:
        inputs = inputs.to(device)
        attention_mask = attention_mask.to(device)
        batch_size, seq_length = inputs.shape

        # Create a mask for randomly selected tokens
        mask = torch.rand(inputs.shape) < masking_prob

        # Randomly replace selected tokens with [MASK] token
        masked_inputs = inputs.clone()
        masked_inputs[mask] = tokenizer.mask_token_id

        pretrain_optimizer.zero_grad()
        outputs = model(input_ids=masked_inputs, attention_mask=attention_mask, decoder_input_ids=inputs)

        # Compute the loss only for the masked tokens
        masked_logits = outputs.logits[mask]
        masked_labels = inputs[mask]
        loss = pretrain_criterion(masked_logits.view(-1, masked_logits.size(-1)), masked_labels.view(-1))

        loss.backward()
        pretrain_optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(pretrain_data)
    print(f"Pretrain Epoch {epoch+1}/{pretrain_epochs} - loss: {epoch_loss:.4f}")

    # Validation
    model.eval()
    val_running_loss = 0.0
    for val_inputs, val_attention_mask in val_loader:
        val_inputs = val_inputs.to(device)
        val_attention_mask = val_attention_mask.to(device)
        batch_size, seq_length = val_inputs.shape

        mask = torch.rand(val_inputs.shape) < masking_prob
        masked_inputs = val_inputs.clone()
        masked_inputs[mask] = tokenizer.mask_token_id

        with torch.no_grad():
            outputs = model(input_ids=masked_inputs, attention_mask=val_attention_mask, decoder_input_ids=val_inputs)
            masked_logits = outputs.logits[mask]
            masked_labels = val_inputs[mask]
            val_loss = pretrain_criterion(masked_logits.view(-1, masked_logits.size(-1)), masked_labels.view(-1))

        val_running_loss += val_loss.item() * val_inputs.size(0)

    epoch_val_loss = val_running_loss / len(val_data)
    print(f"Val Epoch {epoch+1}/{pretrain_epochs} - loss: {epoch_val_loss:.4f}")

    early_stopping(epoch_val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break


Pretrain Epoch 1/10 - loss: 2.0488
Val Epoch 1/10 - loss: 0.3027
Validation loss decreased (inf --> 0.302650).  Saving model ...
Pretrain Epoch 2/10 - loss: 0.1625
Val Epoch 2/10 - loss: 0.0930
Validation loss decreased (0.302650 --> 0.092999).  Saving model ...
Pretrain Epoch 3/10 - loss: 0.0651
Val Epoch 3/10 - loss: 0.0468
Validation loss decreased (0.092999 --> 0.046806).  Saving model ...
Pretrain Epoch 4/10 - loss: 0.0339
Val Epoch 4/10 - loss: 0.0367
Validation loss decreased (0.046806 --> 0.036662).  Saving model ...
Pretrain Epoch 5/10 - loss: 0.0233
Val Epoch 5/10 - loss: 0.0264
Validation loss decreased (0.036662 --> 0.026411).  Saving model ...
Pretrain Epoch 6/10 - loss: 0.0202
Val Epoch 6/10 - loss: 0.0199
Validation loss decreased (0.026411 --> 0.019874).  Saving model ...
Pretrain Epoch 7/10 - loss: 0.0143
Val Epoch 7/10 - loss: 0.0136
Validation loss decreased (0.019874 --> 0.013611).  Saving model ...
Pretrain Epoch 8/10 - loss: 0.0101
Val Epoch 8/10 - loss: 0.0130
Va

For LMA, we first fine-tuned the PLMs on a small amount of task-specific data using a masked language modeling objective. This involves randomly masking some tokens in the input sequence and training the model to predict the masked tokens based on the context provided by the unmasked tokens. This process helps to adapt the PLM to the specific characteristics of the target task and improve its performance on that task.

# Finetuning

In [17]:
# set up the data loader
#train_data = WebNLGDataset(dataset)
batch_size = 32 #32
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

In [18]:
# set up the optimizer and the loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4) #3e-5
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

In [19]:
num_epochs = 2
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=targets)
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(dataset_train)
    print(f"Epoch {epoch+1}/{num_epochs} - loss: {epoch_loss:.4f}")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch 1/2 - loss: 0.9237
Epoch 2/2 - loss: 0.5273


In [ ]:
model.load_state_dict(torch.load('/kaggle/working/checkpoint.pt'))

<All keys matched successfully>

In [ ]:
# Save the entire model
torch.save(model, 'model_T5_flan_small_2020_v2')
print("Model saved successfully.")

Model saved successfully.


In [ ]:
# Load the model
#model = torch.load('/kaggle/input/models/model_T5_flan_small_multi')

# Print a confirmation message
print("Model loaded successfully.")

Model loaded successfully.


# Inference

## are we accounting for the multiple texts targets in the bleu? it doesn't look like it

In [20]:
batch_size=32

In [21]:
from sacrebleu import corpus_bleu
from random import sample
from tqdm import tqdm


# load the WebNLG validation dataset
validation_dataset = load_dataset('web_nlg', 'release_v3.0_en')['test']
validation_dataset = [sample for sample in validation_dataset if sample['lex']['text']] # filter out samples with empty targets
validation_dataset = validation_dataset[:32]
# Select a subset of the validation dataset
#subset_size = 10  # Choose the desired subset size
#validation_subset = sample(list(validation_dataset), subset_size)
validation_data = WebNLGDataset(validation_dataset)

# set up the validation data loader
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

# switch model to evaluation mode
model.eval()

# generate predictions for the validation dataset
predictions = []
references = []
with torch.no_grad():
    for inputs, targets in tqdm(validation_loader, desc='Validation Progress', leave=False):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model.module.generate(inputs, max_length=MAX_TARGET_LENGTH, num_beams=4)
        # convert token IDs to strings
        predicted_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(targets, skip_special_tokens=True)
        # append predicted and target texts for BLEU evaluation
        predictions.extend(predicted_texts)
        references.extend(target_texts)


Generating train split:   0%|          | 0/13211 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1667 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5713 [00:00<?, ? examples/s]

Dataset web_nlg downloaded and prepared to /root/.cache/huggingface/datasets/web_nlg/release_v3.0_en/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Validation Progress:   0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


2
{'category': 'SportsTeam', 'size': 5, 'eid': 'Id1', 'original_triple_sets': {'otriple_set': [['Estádio_Municipal_Coaracy_da_Mata_Fonseca | location | Arapiraca', 'Agremiação_Sportiva_Arapiraquense | league | Campeonato_Brasileiro_Série_C', 'Campeonato_Brasileiro_Série_C | country | Brazil', 'Agremiação_Sportiva_Arapiraquense | nickname | "\'\'Alvinegro"@en', 'Agremiação_Sportiva_Arapiraquense | ground | Estádio_Municipal_Coaracy_da_Mata_Fonseca']]}, 'modified_triple_sets': {'mtriple_set': [['Estádio_Municipal_Coaracy_da_Mata_Fonseca | location | Arapiraca', 'Agremiação_Sportiva_Arapiraquense | league | Campeonato_Brasileiro_Série_C', 'Campeonato_Brasileiro_Série_C | country | Brazil', 'Agremiação_Sportiva_Arapiraquense | nickname | "\'\'Alvinegro"', 'Agremiação_Sportiva_Arapiraquense | ground | Estádio_Municipal_Coaracy_da_Mata_Fonseca']]}, 'shape': '(X (X) (X (X)) (X (X)))', 'shape_type': 'mixed', 'lex': {'comment': ['', ''], 'lid': ['Id1', 'Id2'], 'text': ['Estádio Municipal Coarac

In [ ]:
# Writing predictions to a .txt file
with open("predictions", "w") as f:
    for prediction in predictions:
        f.write(prediction + "\n")

In [ ]:
# calculate BLEU scores
#bleu = corpus_bleu(predictions, [references])

multiple_references = []
for i in range(len(validation_dataset)):
    multiple_references.append(validation_dataset[i]['lex']['text'])

bleu = corpus_bleu(predictions, references)
bleu_multiple = corpus_bleu(predictions, multiple_references)

print(f"BLEU score: {bleu.score}")
print(f"BLEU score with multiple references: {bleu_multiple.score}")

BLEU score: 0.18693009820085577
BLEU score with multiple references: 88.12736030017146


In [ ]:
# Getting the maximum length of the sublists in multiple_references
max_length = max(len(sublist) for sublist in multiple_references)

# Writing multiple_references to separate .txt files
for i in range(max_length):
    with open(f"references{i}", "w") as f:
        for ref_list in multiple_references:
            # Writing the ith element if it exists, otherwise an empty line
            if i < len(ref_list):
                f.write(ref_list[i] + "\n")
            else:
                f.write("\n")

In [ ]:
# calculate BLEU scores
#bleu = corpus_bleu(predictions, [references])

multiple_references = []
for i in range(len(validation_dataset)):
    multiple_references.append(validation_dataset[i]['lex']['text'])

# First, determine the maximum length of sublists
max_len = max(len(refs) for refs in multiple_references)

# Then pad all sublists to that length
padded_references = [refs * (max_len // len(refs)) + refs[:max_len % len(refs)] for refs in multiple_references]

bleu = corpus_bleu(predictions, references)
bleu_multiple = corpus_bleu(predictions, padded_references)

print(f"BLEU score: {bleu.score}")
print(f"BLEU score with padded references: {bleu_multiple.score}")

BLEU score: 0.16214501382472107
BLEU score with padded references: 80.7204465338761


In [ ]:
from datasets import load_metric

metric = load_metric('sacrebleu')

# First, determine the maximum length of sublists
max_len = max(len(refs) for refs in multiple_references)

# Then pad all sublists to that length
padded_references = [refs * (max_len // len(refs)) + refs[:max_len % len(refs)] for refs in multiple_references]

# Now 'padded_references' is a list of lists, where each sublist has the same length.
# We can now compute the SacreBLEU score.

# Note the change in the compute line
score = metric.compute(predictions=predictions, references = padded_references)

print(f"SacreBLEU score: {score['score']}")

SacreBLEU score: 31.99615265661011


In [ ]:
from sacrebleu import corpus_chrf
# Calculate CHR F++ scores
chrf = corpus_chrf(predictions, [references])
chrf_multiple = corpus_chrf(predictions, multiple_references)
print(f"CHR F++ score: {chrf.score}")
print(f"CHR F++ score with multiple references: {chrf_multiple.score}")

CHR F++ score: 54.00287217727219
CHR F++ score with multiple references: 77.50627157244912


In [ ]:
from sacrebleu import corpus_chrf
# Calculate CHR F++ scores
chrf = corpus_chrf(predictions, [references])
chrf_multiple = corpus_chrf(predictions, padded_references)
print(f"CHR F++ score: {chrf.score}")
print(f"CHR F++ score with multiple references: {chrf_multiple.score}")

CHR F++ score: 54.00287217727219
CHR F++ score with multiple references: 70.48380146771281


In [ ]:
!pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_metric
import numpy as np


metric = load_metric('bertscore')

assert len(predictions) == len(references), "The number of predictions and references should be the same."

# Compute the score
score = metric.compute(predictions=predictions, references=references, lang='en')

print(f"BERTScore: {np.mean(score['precision'])}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: 0.9269932945579457


In [22]:
i=0
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(references[i])

[['Estádio_Municipal_Coaracy_da_Mata_Fonseca | location | Arapiraca', 'Agremiação_Sportiva_Arapiraquense | league | Campeonato_Brasileiro_Série_C', 'Campeonato_Brasileiro_Série_C | country | Brazil', 'Agremiação_Sportiva_Arapiraquense | nickname | "\'\'Alvinegro"@en', 'Agremiação_Sportiva_Arapiraquense | ground | Estádio_Municipal_Coaracy_da_Mata_Fonseca']]
----
The Estádio Municipal Coaracy da Mata Fonseca is located in Arapiraca. Agremiaço Sportiva Arapiraquense play in the Campeonato Brasileiro Série C league. The country where Agremiaço Sportiva Arapiraquense play is Brazil. Agremiaço Sportiva Arapiraquense play in the Campeonato Brasileiro Série C league.  Agremiaço Sportiva Arapiraquense play
----



In [23]:
i=1
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(references[i])

[['Nie_Haisheng | birthDate | 1964-10-13', 'Nie_Haisheng | occupation | Fighter_pilot']]
----
Nie Haisheng was born on October 13, 1964. He worked as a fighter pilot.
----



In [24]:
i=2
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(references[i])

[['MotorSport_Vision | locationCity | Fawkham']]
----
The city where MotorSport Vision is located is Fawkham.
----



In [25]:
i=3
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(references[i])

[['Aleksandr_Prudnikov | Person/height | "185.0"^^<http://dbpedia.org/datatype/centimetre>', 'FC_Spartak_Moscow | ground | Otkrytiye_Arena', 'Aleksandr_Prudnikov | team | FC_Spartak_Moscow'], ['Aleksandr_Prudnikov | Person/height | "185.0"^^<http://dbpedia.org/datatype/centimetre>', 'FC_Spartak_Moscow | ground | Otkrytiye_Arena', 'Aleksandr_Prudnikov | clubs | FC_Spartak_Moscow']]
----
Aleksandr Prudnikov's height is 185.0 cm. FC Spartak Moscow's home ground is the Otkrytiye Arena. Aleksandr Prudnikov also plays for FC Spartak Moscow.  Aleksandr Prudnikov's height is 185.0 cm and he plays for FC Spartak Moscow whose home ground is the Otkrytiye Arena.
----



In [26]:
i=4
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(references[i])

[['Ciudad_Ayala | populationMetro | 1777539', 'Ciudad_Ayala | leaderTitle | City Manager', 'Ciudad_Ayala | type | City', 'Ciudad_Ayala | populationDensity | 1604.0', 'Ciudad_Ayala | governmentType | Council-manager_government']]
----
The population of Ciudad Ayala is 1777539. The leader title of Ciudad Ayala is "City Manager". Ciudad Ayala is a city. Ciudad Ayala has a population density of 1604.0. The government type of Ciudad Ayala is Council-Manager Government.  The leader title of Ciudad Ayala is "City Manager". The population density of Ciudad Ayala is 1604.0. The government type of Ciudad Ayala is Council-Manager
----



In [27]:
i=5
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(references[i])

[['Bootleg_Series_Volume_1:_The_Quine_Tapes | previousWork | Squeeze_(The_Velvet_Underground_album)', 'Squeeze_(The_Velvet_Underground_album) | subsequentWork | 1969:_The_Velvet_Underground_Live']]
----
The Bootleg Series Volume 1: The Queen Tapes was preceded by Squeeze (The Velvet Underground album). The Squeeze (The Velvet Underground album) was followed by 1969: The Velvet Underground Live. The Queen Tapes was also preceded by Squeeze (The Velvet Underground album).  The Queen Tapes was preceded by Squeeze (The Velvet Underground album) and 1969: The Velvet Underground Live.
----



In [29]:
i=6
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(references[i])

[['Ciudad_Ayala | populationDensity | 1604.0', 'Ciudad_Ayala | leaderTitle | Governator', 'Ciudad_Ayala | country | Mexico', 'Ciudad_Ayala | elevation | 1147.0', 'Ciudad_Ayala | timeZone | Pacific_Daylight_Time']]
----
The population density of Ciudad Ayala is 1604.0. The leader title of Ciudad Ayala is Governator. Ciudad Ayala is located in Mexico. Ciudad Ayala is 1147.0 above sea level. The time zone of Ciudad Ayala is Pacific Daylight Time. Ciudad Ayala is located in the Pacific Time Zone.  Ciudad Ayala is located in Mexico and has a population density of 1604.0. The leader title of Ciudad Ayala is a
----



In [31]:
i=8
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print('----')
print(predictions[i])
print('----')
print(references[i])

[['Saint_Petersburg | foundingDate | 1703-05-27', 'Olga_Bondareva | deathPlace | Saint_Petersburg', 'Saint_Petersburg | areaTotal | 1439.0']]
----
The founding date of Saint Petersburg is 1703-05-27. Olga Bondareva died in Saint Petersburg. The total area of Saint Petersburg is 1439.0. Olga Bondareva was born in Saint Petersburg.  Olga Bondareva was born in Saint Petersburg on 27th May 1703 and died in Saint Petersburg. The total area of Saint Petersburg is 1439.0.
----

